In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/000GithubRepos/000recsys_key_papers_implementation/SASRec')

In [3]:
import time
import torch
import argparse
from model import SASRec
from utils import *

In [4]:
def str2bool(s):
    if s not in {'false', 'true'}:
        raise ValueError('Not a valid boolean string')
    return s == 'true'

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', required=True)
    parser.add_argument('--train_dir', required=True)
    parser.add_argument('--batch_size', default=128, type=int)
    parser.add_argument('--lr', default=0.001, type=float)
    parser.add_argument('--maxlen', default=50, type=int)
    parser.add_argument('--hidden_units', default=50, type=int)
    parser.add_argument('--num_blocks', default=2, type=int)
    parser.add_argument('--num_epochs', default=201, type=int)
    parser.add_argument('--num_heads', default=1, type=int)
    parser.add_argument('--dropout_rate', default=0.5, type=float)
    parser.add_argument('--l2_emb', default=0.0, type=float)
    parser.add_argument('--device', default='cpu', type=str)
    parser.add_argument('--inference_only', default=False, type=str2bool)
    parser.add_argument('--state_dict_path', default=None, type=str)
    return parser.parse_args(args=['--dataset', '../data/ml-1m/ml-1m.txt', '--train_dir', 'default'])
args = get_args()

In [5]:
args.dataset

'../data/ml-1m/ml-1m.txt'

In [6]:
args.train_dir

'default'

In [7]:
# [str(k) + ',' + str(v) 

for k, v in sorted(vars(args).items(), key=lambda x: x[0]):
    print(f"{k:>20} : {str(v):20}")

          batch_size : 128                 
             dataset : ../data/ml-1m/ml-1m.txt
              device : cpu                 
        dropout_rate : 0.5                 
        hidden_units : 50                  
      inference_only : False               
              l2_emb : 0.0                 
                  lr : 0.001               
              maxlen : 50                  
          num_blocks : 2                   
          num_epochs : 201                 
           num_heads : 1                   
     state_dict_path : None                
           train_dir : default             


In [8]:
if not os.path.isdir(args.dataset + '_' + args.train_dir):  # 데이터 없다면 폴더 만듦
    os.makedirs(args.dataset + '_' + args.train_dir)
    print(f"made {args.dataset + '_' + args.train_dir} folder")
with open(os.path.join(args.dataset + '_' + args.train_dir, 'args.txt'), 'w') as f:  # argument 저장
    f.write('\n'.join([str(k) + ',' + str(v) for k, v in sorted(vars(args).items(), key=lambda x: x[0])]))
    print(f"wrote '{os.path.join(args.dataset + '_' + args.train_dir, 'args.txt')}'")
f.close()

wrote '../data/ml-1m/ml-1m.txt_default/args.txt'


In [9]:
from collections import defaultdict

# train/val/test data generation
def data_partition(fname):
    usernum = 0
    itemnum = 0
    User = defaultdict(list)
    user_train = {}
    user_valid = {}
    user_test = {}
    # assume user/item index starting from 1
    f = open(f'{fname}')
    for line in f:
        u, i = line.rstrip().split(' ')
        u = int(u)
        i = int(i)
        usernum = max(u, usernum)
        itemnum = max(i, itemnum)
        User[u].append(i)

    for user in User:
        nfeedback = len(User[user])
        if nfeedback < 3:
            user_train[user] = User[user]
            user_valid[user] = []
            user_test[user] = []
        else:
            user_train[user] = User[user][:-2]
            user_valid[user] = []
            user_valid[user].append(User[user][-2])
            user_test[user] = []
            user_test[user].append(User[user][-1])
    return [user_train, user_valid, user_test, usernum, itemnum]

dataset = data_partition(args.dataset)

In [10]:
[user_train, user_valid, user_test, usernum, itemnum] = dataset

image.png

In [11]:
type(user_train)

dict

In [12]:
print(f"유저 최소값 : {min(user_train.keys())}")
print(f"유저 최대값 : {max(user_train.keys()):,}")
print(f"유저 수 : {len(user_train):,}")

유저 최소값 : 1
유저 최대값 : 6,040
유저 수 : 6,040


In [13]:
user_valid[1]

[78]

In [14]:
num_batch = len(user_train) // args.batch_size # tail? + ((len(user_train) % args.batch_size) != 0)
cc = 0.0
for u in user_train:
    cc += len(user_train[u])

print(f"len(user_train) : {len(user_train)}")
print(f"args.batch_size : {args.batch_size}")
print(f"num_batch : {num_batch}")
print(f"average sequence length : {(cc / len(user_train)):.2f}")

len(user_train) : 6040
args.batch_size : 128
num_batch : 47
average sequence length : 163.50


In [15]:
f = open(os.path.join(args.dataset + '_' + args.train_dir, 'log.txt'), 'w')
print(f"logging 위치 : {os.path.join(args.dataset + '_' + args.train_dir, 'log.txt')}")

logging 위치 : ../data/ml-1m/ml-1m.txt_default/log.txt


In [42]:
sampler = WarpSampler(user_train, usernum, itemnum, batch_size=args.batch_size, maxlen=args.maxlen, n_workers=3)
model = SASRec(usernum, itemnum, args).to(args.device) # no ReLU activation in original SASRec implementation?

In [17]:
model

SASRec(
  (item_emb): Embedding(3417, 50, padding_idx=0)
  (pos_emb): Embedding(50, 50)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (attention_layernorms): ModuleList(
    (0-1): 2 x LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0-1): 2 x MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
    )
  )
  (forward_layernorms): ModuleList(
    (0-1): 2 x LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (forward_layers): ModuleList(
    (0-1): 2 x PointWiseFeedForward(
      (conv1): Conv1d(50, 50, kernel_size=(1,), stride=(1,))
      (dropout1): Dropout(p=0.5, inplace=False)
      (relu): ReLU()
      (conv2): Conv1d(50, 50, kernel_size=(1,), stride=(1,))
      (dropout2): Dropout(p=0.5, inplace=False)
    )
  )
  (last_layernorm): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
)

In [36]:
for i, (name, param) in enumerate(model.named_parameters()):
    try:
        torch.nn.init.xavier_normal_(param.data)
        print(f"{i:3}\t{name:50} : {param.shape}")
    except:
        pass # just ignore those failed init layers
        print(f"{i:3}\t{name:50} : {param.shape}\t<-- failed")

  0	item_emb.weight                                    : torch.Size([3417, 50])
  1	pos_emb.weight                                     : torch.Size([50, 50])
  2	attention_layernorms.0.weight                      : torch.Size([50])	<-- failed
  3	attention_layernorms.0.bias                        : torch.Size([50])	<-- failed
  4	attention_layernorms.1.weight                      : torch.Size([50])	<-- failed
  5	attention_layernorms.1.bias                        : torch.Size([50])	<-- failed
  6	attention_layers.0.in_proj_weight                  : torch.Size([150, 50])
  7	attention_layers.0.in_proj_bias                    : torch.Size([150])	<-- failed
  8	attention_layers.0.out_proj.weight                 : torch.Size([50, 50])
  9	attention_layers.0.out_proj.bias                   : torch.Size([50])	<-- failed
 10	attention_layers.1.in_proj_weight                  : torch.Size([150, 50])
 11	attention_layers.1.in_proj_bias                    : torch.Size([150])	<-- failed
 12	atten

In [48]:
model.named_parameters().__next__()

('item_emb.weight',
 Parameter containing:
 tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.3141, -1.1014,  1.3818,  ..., -2.1240,  2.4541, -0.4215],
         [-0.3108,  0.3836, -0.1435,  ...,  1.0293,  0.4524, -2.5776],
         ...,
         [-0.2907,  0.3693, -0.5840,  ...,  0.2404, -1.0962, -1.7210],
         [-0.3701,  0.3648,  1.7048,  ..., -0.6726, -0.3195,  0.7800],
         [ 0.1361,  0.3828, -1.1043,  ...,  0.1696,  0.6365, -1.0599]],
        requires_grad=True))

In [20]:
name

'last_layernorm.bias'

In [34]:
param.data.

torch.Size([50])

In [23]:
from torch import nn

In [29]:
w = torch.empty(3, 5)
print(f"w : {w}")
nn.init.xavier_normal_(w)
print(f"w : {w}")

w : tensor([[2.1707e-18, 7.0952e+22, 1.7748e+28, 1.8176e+31, 7.2708e+31],
        [5.0778e+31, 3.2608e-12, 1.7728e+28, 7.0367e+22, 2.1715e-18],
        [6.7572e-07, 1.7092e-04, 1.7079e-07, 2.6219e-09, 7.9965e+20]])
w : tensor([[-0.5716, -0.6212, -0.1203,  1.0478, -0.2580],
        [-0.2840,  0.0770, -0.0368,  0.8086,  0.0777],
        [ 0.3240, -0.4930,  0.3078,  0.5676,  0.7438]])


In [33]:
w.mean()

tensor(0.1046)

In [24]:
epoch_start_idx = 1
if args.state_dict_path is not None:
    try:
        model.load_state_dict(torch.load(args.state_dict_path, map_location=torch.device(args.device)))
        tail = args.state_dict_path[args.state_dict_path.find('epoch=') + 6:]
        epoch_start_idx = int(tail[:tail.find('.')]) + 1
    except: # in case your pytorch version is not 1.6 etc., pls debug by pdb if load weights failed
        print('failed loading state_dicts, pls check file path: ', end="")
        print(args.state_dict_path)
        print('pdb enabled for your quick check, pls type exit() if you do not need it')
        import pdb; pdb.set_trace()

In [25]:
if args.inference_only:
    model.eval()
    t_test = evaluate(model, dataset, args)
    print('test (NDCG@10: %.4f, HR@10: %.4f)' % (t_test[0], t_test[1]))

In [26]:
# ce_criterion = torch.nn.CrossEntropyLoss()
# https://github.com/NVIDIA/pix2pixHD/issues/9 how could an old bug appear again...
bce_criterion = torch.nn.BCEWithLogitsLoss() # torch.nn.BCELoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.98))

T = 0.0
t0 = time.time()

In [27]:
u

6040

In [28]:
num_batch

47

# Wandb 설정

In [29]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=2ba426bc65a19a64181d2a731ea2830b5f1f0bfa83659b98cbef9aaa8b905de4
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [30]:
import wandb

In [31]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 10


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: ignored

In [ ]:
model_name = 'SASRec'

In [ ]:
# 🐝 1️⃣ Start a new run to track this script
wandb.init(
    # Set the project where this run will be logged
    project="recsys_key_papers_implementation", 
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"{model_name}", 
    # Track hyperparameters and run metadata
    config=vars(args))

# 학습

In [ ]:
for epoch in range(epoch_start_idx, args.num_epochs + 1):
    if args.inference_only: break # just to decrease identition
    for step in range(num_batch): # tqdm(range(num_batch), total=num_batch, ncols=70, leave=False, unit='b'):
        u, seq, pos, neg = sampler.next_batch() # tuples to ndarray
        u, seq, pos, neg = np.array(u), np.array(seq), np.array(pos), np.array(neg)
        pos_logits, neg_logits = model(u, seq, pos, neg)
        pos_labels, neg_labels = torch.ones(pos_logits.shape, device=args.device), torch.zeros(neg_logits.shape, device=args.device)
        # print("\neye ball check raw_logits:"); print(pos_logits); print(neg_logits) # check pos_logits > 0, neg_logits < 0
        adam_optimizer.zero_grad()
        indices = np.where(pos != 0)
        loss = bce_criterion(pos_logits[indices], pos_labels[indices])
        loss += bce_criterion(neg_logits[indices], neg_labels[indices])
        for param in model.item_emb.parameters(): loss += args.l2_emb * torch.norm(param)
        loss.backward()
        adam_optimizer.step()
    # print("loss in epoch {} iteration {}: {}".format(epoch, step, loss.item())) # expected 0.4~0.6 after init few epochs
    

    # if epoch % 20 == 0:
    model.eval()
    t1 = time.time() - t0
    T += t1
    print('Evaluating', end='')
    t_test = evaluate(model, dataset, args)
    t_valid = evaluate_valid(model, dataset, args)
    print(f"epoch:{epoch:4}, time: {T//3600:.0f}h {T//60:.0f}min {T%60:.0f}sec, valid (NDCG@10: {t_valid[0]:.4f}, HR@10: {t_valid[1]:.4f}), test (NDCG@10: {t_test[0]:.4f}, HR@10: {t_test[1]:.4f})")
    wandb.log({"epoch": epoch, "loss": loss.item(), "valid NDCG@10" : t_valid[0], "valid HR@10" : t_valid[1], "test NDCG@10" : t_test[0], "test HR@10" : t_test[1]})
    f.write(str(t_valid) + ' ' + str(t_test) + '\n')
    f.flush()
    t0 = time.time()
    model.train()

    if epoch == args.num_epochs:
        folder = args.dataset + '_' + args.train_dir
        fname = 'SASRec.epoch={}.lr={}.layer={}.head={}.hidden={}.maxlen={}.pth'
        fname = fname.format(args.num_epochs, args.lr, args.num_blocks, args.num_heads, args.hidden_units, args.maxlen)
        torch.save(model.state_dict(), os.path.join(folder, fname))

Evaluating........................................................................................................................epoch:   1, time: 0.0h 0.0min 18.05775785446167sec, valid (NDCG@10: 0.2384, HR@10: 0.4402), test (NDCG@10: 0.2482, HR@10: 0.4445)
Evaluating........................................................................................................................epoch:   2, time: 0.0h 0.0min 19.942630767822266sec, valid (NDCG@10: 0.2455, HR@10: 0.4440), test (NDCG@10: 0.2487, HR@10: 0.4487)
Evaluating........................................................................................................................epoch:   3, time: 0.0h 0.0min 21.87635374069214sec, valid (NDCG@10: 0.2468, HR@10: 0.4478), test (NDCG@10: 0.2495, HR@10: 0.4515)
Evaluating........................................................................................................................epoch:   4, time: 0.0h 0.0min 23.80896019935608sec, valid (NDCG@10: 0.2490, HR@10: 0.4540